# WhoScorred Match-Centre Scraping - Part 02

In this part, I shall be scraping the shots data from each of WhoScored match URLs I had scraped earlier. Essentially, I run a loop function which goes through each & every match URL in the excel file and picks out just the shots data that I want. 

I want to give a massive massive shout-out to the Github User [Ali Hasan Khan](https://github.com/Ali-Hasan-Khan/Scrape-Whoscored-Event-Data) because it is primiarly his code that I have tweaked for my benefit. There is a separate python scripts of his that is used in this notebook called "main" which should be attached along with the notebooks.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
import main
import seaborn as sns
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import datetime
import re
import os

In [ ]:
### Please import the excel file named 'PL 2013-14 -- WhoScored Match URLs' under the name "match_urls"

season = '2013-14.xlsx'

import_folder_path = 'C:/Users/tharu/OneDrive/Desktop/Big-PL-Project/PL-Project/WhoScored Match URLs/Final/Premier League' 
import_file_path = f"{import_folder_path}/{season}"

export_folder_path = 'C:/Users/tharu/OneDrive/Desktop/Big-PL-Project/PL-Project/WhoScored Shots Data - Raw/Premier League' 

match_urls = pd.read_excel(import_file_path)
match_urls

In [ ]:
### Picking out the attributes from WhoScored I would like to have for each & every shot I scrape

final_df = pd.DataFrame(columns=['minute', 'second', 'expandedMinute',
   'teamId', 'h_a', 'x', 'y','period','type', 'playerId','playerName',
   'relatedPlayerId','blockedX', 'blockedY','goalMouthZ','goalMouthY', 'isGoal', 
   'shotBodyType','situation', 'shotSixYardBox', 'shotPenaltyArea', 'shotOboxTotal',
   'shotOpenPlay', 'shotCounter', 'shotSetPiece', 'shotDirectCorner',                 
   'shotOffTarget','shotOnPost','shotOnTarget','shotBlocked','goalOwn',             
   'bigChanceMissed','bigChanceScored','bigChanceCreated',
   'penaltyScored','penaltyMissed',
   'penaltyShootoutScored', 'penaltyShootoutMissedOffTarget', 'penaltyShootoutSaved', 
    'start_date', 'home_teamId', 'away_teamId', 'matchId', 'season', 'match_count'])

In [ ]:
### Please ensure you have the correct chromedriver application! If not, download it 
### Be ready to wait haha; It will take at least two & a half hours at the very least
### for this loop function to go through all 380 match URLs.

for i in range(len(match_urls)):
    
    if __name__ == "__main__":
        service = Service(executable_path=r'C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\chromedriver.exe')
        options = webdriver.ChromeOptions()
        driver = webdriver.Chrome(service=service, options=options)   
    
    # whoscored match centre url of the required match
    url = match_urls['Links'][i]
    match_data = main.getMatchData(driver, url)

    # Match dataframe containing info about the match
    matches_df = main.createMatchesDF(match_data)

    # Events dataframe   
    events_df = main.createEventsDF(match_data)
    shots = events_df.query('isShot == True')
    
    shots_df = shots[['minute', 'second', 'expandedMinute',
   'teamId', 'h_a', 'x', 'y','period','type', 'playerId','playerName',
   'relatedPlayerId','blockedX', 'blockedY','goalMouthZ','goalMouthY', 'isGoal', 
   'shotBodyType','situation', 'shotSixYardBox', 'shotPenaltyArea', 'shotOboxTotal',
   'shotOpenPlay', 'shotCounter', 'shotSetPiece', 'shotDirectCorner',                 
   'shotOffTarget','shotOnPost','shotOnTarget','shotBlocked','goalOwn',             
   'bigChanceMissed','bigChanceScored','bigChanceCreated',
   'penaltyScored','penaltyMissed',
   'penaltyShootoutScored', 'penaltyShootoutMissedOffTarget', 'penaltyShootoutSaved']]
    
    
    start_date = datetime.strptime(matches_df.iloc[0, matches_df.columns.get_loc('startDate')].split("T")[0],'%Y-%m-%d').date()
    home_teamId = int(re.findall(r'\d+', str(matches_df.iloc[0, matches_df.columns.get_loc('home')]).split(",")[0])[0])
    away_teamId = int(re.findall(r'\d+', str(matches_df.iloc[0, matches_df.columns.get_loc('away')]).split(",")[0])[0])
    matchId = int(re.findall(r'\d+', str(events_df.iloc[0, events_df.columns.get_loc('matchId')]).split(",")[0])[0])

    shots_df['start_date'] = start_date
    shots_df['home_teamId'] = home_teamId
    shots_df['away_teamId'] = away_teamId
    shots_df['matchId'] = matchId
    shots_df['match_count'] = i

    Players = events_df.drop_duplicates(['playerId','playerName'])[['playerId','playerName']].dropna()
    Players = Players.rename(columns={'playerId': 'relatedPlayerId', 'playerName': 'relatedplayerName'})
    Players['relatedPlayerId'] = Players['relatedPlayerId'].astype(float)

    shots_df_final = shots_df.merge(Players, on='relatedPlayerId', how='left')
    final_df = pd.concat([final_df, shots_df_final], ignore_index=True)

In [ ]:
final_df

In [ ]:
### Exporting the raw shots data to the folder of choice
final_df.to_excel(os.path.join(export_folder_path, season), index=False)